# Modificar Valores (Escribir en OPC UA)

Un script muy sencillo para conectarse al PLC y modificar directamente el valor de las variables. Simplemente ejecuta la primera celda de configuración y luego usa la segunda para escribir los datos.

In [1]:
import asyncio
from asyncua import Client, ua

ENDPOINT = "opc.tcp://192.168.0.20:4840"
FOLDER_NAME = "MES"
variable_recibida = 1
async def find_node(client, folder_name, var_name, max_depth=5):
    """Busca recursivamente la carpeta y luego la variable."""
    async def walk(node, depth):
        if depth > max_depth: return None
        try: children = await node.get_children()
        except: return None
        
        for child in children:
            try: bname = (await child.read_browse_name()).Name
            except: continue
            
            if bname == folder_name:
                for var in await child.get_children():
                    if (await var.read_browse_name()).Name == var_name:
                        return var
            
            found = await walk(child, depth + 1)
            if found: return found
        return None
        
    return await walk(client.nodes.objects, 0)

async def write_opcua(var_name, value, variant_type=None):
    """Se conecta al servidor y escribe el valor proporcionado."""
    async with Client(url=ENDPOINT) as client:
        node = await find_node(client, FOLDER_NAME, var_name)
        if not node:
            print(f"❌ No se encontró la variable: {var_name}")
            return
            
        # Escribimos el valor (usando el tipo de variante si se especifica)
        if variant_type:
            dv = ua.DataValue(ua.Variant(value, variant_type))
            await node.write_value(dv)
        else:
            await node.write_value(value) # Inferencia automática
            
        print(f"✅ ¡Éxito! Variable '{var_name}' actualizada a: {value}")

await write_opcua("Machine_State", variable_recibida, ua.VariantType.Int16)

Requested session timeout to be 3600000ms, got 30000ms instead


❌ No se encontró la variable: Machine_State
